In [3]:
import os

from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import display, Image


os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"


In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)


class State(MessagesState):
    summary: str


def call_model(state: State):
    summary = state.get("summary", "")

    if summary:
        system_message = SystemMessage(content=f"Summary of conversation earlier: {summary}")
        messages = [system_message] + state["messages"]
    else:
        messages = state["messages"]

    response = llm.invoke(messages)
    return {"messages": [response]}


def summarize_conversation(state: State):
    summary = state.get("summary", "")

    if summary:
        summary_message = HumanMessage(content=(
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        ))
    else:
        summary_message = HumanMessage(content="Create a summary of the conversation above:")

    messages = state["messages"] + [summary_message]
    response = llm.invoke(messages)

    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


def should_continue(state: State):
    messages = state["messages"]
    if len(messages) > 6:
        return "summarize_conversation"
    return END


In [8]:
workflow = StateGraph(State)

workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


In [9]:
config = {"configurable": {"thread_id": "1"}}

input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hi Lance! 👋 How’s it going? What’s on your mind today?
================================== Ai Message ==================================

Your name is Lance! 😊 You just told me—did I get it right?
================================== Ai Message ==================================

That's awesome, Lance! 🏈 The San Francisco 49ers are a legendary team with a rich history. Are you excited about their season? Who’s your favorite player—past or present? Joe Montana, Jerry Rice, or maybe someone from the current roster?


In [ ]:
graph.get_state(config).values.get("summary", "")

''

In [11]:
input_message = HumanMessage(content="i like Nick Bosa, isn't he the highest paid defensive player?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Yes, you're absolutely right, Lance! Nick Bosa became the **highest-paid defensive player in NFL history** after signing a massive contract extension with the 49ers in September 2023. His deal is worth **$170 million over five years**, with **$122.5 million guaranteed**! 💰🔥

Bosa is an absolute beast on the field—his pass-rushing skills and dominance make him a cornerstone of the 49ers' defense. No wonder you're a fan! Are you pumped to see him wreak havoc this season?


In [13]:
graph.get_state(config).values.get("summary", "")

"Sure! Here's a summary of our conversation:\n\nLance introduced himself and shared that he's a fan of the San Francisco 49ers. He mentioned that he likes Nick Bosa, and we discussed how Bosa recently became the highest-paid defensive player in NFL history with a $170 million contract extension. Lance is excited about Bosa's impact on the team this season! 🏈"